In [1]:
from owlready import *
from owlready.editor import *
from owlready.instance_editor import *
import types
import editobj3

* Owlready * Creating new ontology owl <http://www.w3.org/2002/07/owl>.
* Owlready * Creating new ontology 22-rdf-syntax-ns <http://www.w3.org/1999/02/22-rdf-syntax-ns>.
* Owlready * Creating new ontology rdf-schema <http://www.w3.org/2000/01/rdf-schema>.
* Owlready * Creating new ontology XMLSchema <http://www.w3.org/2001/XMLSchema>.
* Owlready * Creating new ontology anonymous <http://anonymous>.
* Owlready * Creating new ontology owlready_ontology <http://www.lesfleursdunormal.fr/static/_downloads/owlready_ontology.owl>.
* Owlready *     ...loading ontology owlready_ontology from /home/rana/Envs/amazon3/lib/python3.5/site-packages/owlready/owlready_ontology.owl...


In [2]:
onto = Ontology("http://test.org/onto.owl")

* Owlready * Creating new ontology onto <http://test.org/onto.owl>.


In [ ]:
def retrieve_info_from_page(asin):

    item_characteristics_dict = {}

    page = open('./pages/'+asin, 'r')
    soup = BeautifulSoup(page.read(), 'html.parser')
    tab = soup.find('div', {'id': 'HLCXComparisonWidget_feature_div'})

    item_characteristics_dict['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
    item_characteristics_dict['asin'] = asin
    
    bullets = soup.find('div', {'id':'feature-bullets'})
    #for span in bullets.findAll('span'):
        #print span.text.strip()

    try:
        for i, tr in enumerate(tab.findAll('tr')):
            # tr 0:
            # tr 1:
            # tr 2:
            # tr 3: customer rating
            if i<4: continue

            key = tr.findAll('td')[0].span.text
            if tr.findAll('td')[1].span != None: value = tr.findAll('td')[1].span.text
            else: value = tr.findAll('td')[1].a.text
            item_characteristics_dict[key.strip().lower()] = value.strip().lower()
    except: 
        print 'no tr tab'
        
    # PROD DETAILS
    details_tab = soup.find('div', {'id':'prodDetails'})
    #print details_tab

    if soup.find('div', {'id':'productDescription'})!=None: 
        description1 = soup.find('div', {'id':'productDescription'}).p.text
        print description1

    pprint(item_characteristics_dict, indent=4)
    
    review_list = []
    f = open('./results/' + asin[:-5] + '.json', 'r')
    for line in f:
        review_list.append(json.loads(line)['text'])
        
    rel_list = []
    for r in review_list:
        s = parsetree(r.encode('ascii', 'ignore'), relations=True, lemmata=True)
        for sentence in s:
            rel_dict = {}
        
            for rel in sentence.relations:
                for rnum in sentence.relations[rel]:

                    if rnum not in rel_dict: 
                        rel_dict[rnum] = {}
                        rel_dict[rnum][rel] = sentence.relations[rel][rnum]
                    else:
                        rel_dict[rnum][rel] = sentence.relations[rel][rnum]

            rel_list.append(rel_dict)
    
    return item_characteristics_dict, rel_list

In [3]:
class Phone(Thing):
    ontology = onto
    
class Camera(Thing):
    ontology = onto
    
class has_camera(Property):
    ontology = onto
    domain = [Phone]
    range = [Camera]
ANNOTATIONS[has_camera]["python_name"] = "has_camera"

camera_8mp = Camera("camera_8mp")
camera_16mp = Camera("camera_16mp")

class galaxy_s8 (Phone):
    ontology = onto
    equivalent_to = [Phone & restriction(has_camera, VALUE, camera_16mp)]
    
    def print_name(self):
        print("hey there")

In [4]:
query = Phone(has_camera = [camera_16mp])
query.closed_world()
onto.sync_reasoner()

* Owlready * Running HermiT...
    java -Xmx2000M -cp /home/rana/Envs/amazon3/lib/python3.5/site-packages/owlready/hermit:/home/rana/Envs/amazon3/lib/python3.5/site-packages/owlready/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpfj7ghuzj
* Owlready * HermiT took 0.485062837600708 seconds
* Owlready * Reparenting phone_1: {onto.Phone} => {onto.galaxy_s8}


In [5]:
query.print_name()

hey there


In [6]:
for ontology in onto.indirectly_imported_ontologies(): configure_editobj_from_ontology(ontology)

editobj3.GUI = "HTML" # "Qt" or "HTML"

if editobj3.GUI == "Qt": # Required by Qt
    import sys, PyQt5.QtWidgets as qtwidgets
    if qtwidgets.QApplication.startingUp(): qtwidgets.app = qtwidgets.QApplication(sys.argv)

editor = OntologyInstanceEditor()
editor.set_ontology(onto, edited_classes = [Phone, Camera, galaxy_s8, Thing])
editor.main()

* Editobj3 * Editor dialog ready at http://127.0.0.1:8080/editobj/main.html !
======== Running on http://127.0.0.1:8080 ========
(Press CTRL+C to quit)


ImportError: No module named 'pillow'